SQLite Database Initialization
This section initializes the SQLite database by creating necessary tables such as Items, Auctions, and ActionEvents. It ensures the existence of the required tables and sets up the database schema.

File and Configuration Setup
Additionally, it creates other essential files and configurations needed for subsequent operations.

In [ ]:
import os
import sys
import sqlite3
import json
import pandas as pd
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from tqdm import tqdm
from pathlib import Path

wd = Path(os.path.dirname(os.path.abspath("__file__"))).parent.resolve()
sys.path.append(str(wd))

from data.transformers import add_features, transform_data

In [ ]:
db_path = 'auction.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Items (
        item_id INT PRIMARY KEY,
        item_name TEXT,
        quality TEXT,
        item_level INT,
        required_level INT,
        item_class TEXT,
        item_subclass TEXT,
        purchase_price_gold INT,
        purchase_price_silver INT,
        sell_price_gold INT,
        sell_price_silver INT,
        max_count INT,
        is_stackable INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Auctions (
        auction_id INT PRIMARY KEY,
        bid INT,
        buyout INT,
        quantity INT,
        time_left TEXT,
        item_id INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS ActionEvents (
        auction_id INT,
        record DATETIME,
        PRIMARY KEY (auction_id, record),
        FOREIGN KEY (auction_id) REFERENCES Auctions(auction_id)
    )
''')

conn.commit()
conn.close()

In [ ]:
file_info = {}
data_dir = 'sample/'

for root, dirs, files in os.walk(data_dir):
    for filename in tqdm(files):
        filepath = os.path.join(root, filename)
        date = datetime.strptime(filename.split('.')[0], '%Y%m%dT%H')

        file_info[filepath] = date

file_info = {k: v for k, v in sorted(file_info.items(), key=lambda item: item[1])}
filenames = list(file_info.keys())

filenames[:5]

MySQL Items Data Retrieval
In this part, the script retrieves data from a MySQL database. It reads the MySQL database configuration from a JSON file, establishes a connection, and fetches data from the Items table. The retrieved data is then loaded into a Pandas DataFrame.

SQLite Database Update
After retrieving the data, the script connects to the SQLite database, deletes all existing records from the Items table, and appends the newly fetched data.

In [ ]:
db_path = 'auction.db'
data_dir = 'sample/'
db = sqlite3.connect(db_path)
cursor = db.cursor()
auction_ids = {}

for i, filepath in tqdm(enumerate(filenames)):
    print(f"Processing file {filepath} ({i+1}/{len(filenames)})")
    
    try:
        data = json.load(open(filepath, "r"))
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error reading file {filepath}: {e}")
        continue

    filename = filepath.split('/')[-1]
    auction_record = datetime.strptime(filename[:-5], "%Y%m%dT%H")

    if i == 0:
        auctions_data = []

        for auction in data["auctions"]:
            if auction["id"] not in auction_ids:
                auctions_data.append((auction["id"], auction["bid"], auction["buyout"], auction["quantity"], auction["time_left"], auction["item"]["id"]))
                auction_ids.update({auction["id"]: 1})

        try:
            cursor.executemany("""
                INSERT INTO Auctions (auction_id, bid, buyout, quantity, time_left, item_id)
                VALUES (?, ?, ?, ?, ?, ?)
            """, auctions_data)
        except sqlite3.Error as err:
            db.rollback()
            print(f"Error inserting auction data for file {filepath} in Auctions: {err}")

    action_events_data = []
    for auction in data["auctions"]:
        if auction["id"] in auction_ids:
            action_events_data.append((auction["id"], auction_record.strftime('%Y-%m-%d %H:%M:%S')))
            
    try:
        cursor.executemany("""
            INSERT OR REPLACE INTO ActionEvents (auction_id, record)
            VALUES (?, ?)
        """, action_events_data)
    except sqlite3.Error as err:
        db.rollback()
        print(f"Error inserting auction events for file {filepath} in ActionEvents: {err}")

db.commit()
cursor.close()
db.close()

Auction Data and Items Storage
This section processes JSON files containing auction data. It iterates through the files, extracts relevant information, and inserts it into the Auctions and ActionEvents tables of the SQLite database.

Data Import from Pandas DataFrame
Moreover, it initializes a connection to the SQLite database and imports data from a Pandas DataFrame into the Items table. This DataFrame is generated from MySQL database retrieval, ensuring the SQLite Items table is up-to-date.

In [ ]:
items_df = pd.read_csv('items.csv')
items = [tuple(x) for x in items_df.values]

try:
    db = sqlite3.connect(db_path)
    print("Connected to SQLite")
except sqlite3.Error as err:
    print(f"Error connecting to SQLite: {err}")

cursor = db.cursor()

cursor.execute('''
    DROP TABLE IF EXISTS Items
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Items (
        item_id INT PRIMARY KEY,
        item_name TEXT,
        quality TEXT,
        item_level INT,
        required_level INT,
        item_class TEXT,
        item_subclass TEXT,
        purchase_price_gold INT,
        purchase_price_silver INT,
        sell_price_gold INT,
        sell_price_silver INT,
        max_count INT,
        is_stackable INT
    )
''')

try:
    print(items[0])
    cursor.executemany("""
        INSERT OR REPLACE INTO Items (item_id, item_name, quality, item_level, required_level, item_class, item_subclass, purchase_price_gold, purchase_price_silver, sell_price_gold, sell_price_silver, max_count, is_stackable)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, items)
    db.commit()
    print("Inserted items into SQLite: " + str(len(items)))
except sqlite3.Error as err:
    print(f"Error executing SQL query: {err}")

finally:
    cursor.close()
    db.close()

# Read auction.db
It is responsible for connecting to an SQLite database, executing a SQL query involving multiple tables, and retrieving the results. These results are stored in the variable

In [ ]:
db_path = 'auction.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

query = """
    SELECT
        a.auction_id,
        a.bid / 10000.0 AS bid_in_gold,
        a.buyout / 10000.0 AS buyout_in_gold,
        (a.buyout / 10000.0) / a.quantity AS unit_price,
        a.quantity,
        a.time_left,
        a.item_id,
        i.item_name,
        i.quality,
        i.item_class,
        i.item_subclass,
        i.is_stackable,
        i.purchase_price_gold,
        i.required_level,
        i.item_level,
        i.sell_price_gold,
        MIN(ae.record) AS first_appearance_timestamp,
        strftime('%Y', MIN(ae.record)) AS first_appearance_year,
        strftime('%m', MIN(ae.record)) AS first_appearance_month,
        strftime('%d', MIN(ae.record)) AS first_appearance_day,
        strftime('%H', MIN(ae.record)) AS first_appearance_hour,
        COUNT(*) AS hours_on_sale
    FROM Auctions a
    JOIN ActionEvents ae ON a.auction_id = ae.auction_id
    JOIN Items i ON i.item_id = a.item_id
    WHERE A.time_left <> 'SHORT'
    GROUP BY a.auction_id
"""

cursor.execute(query)
results = cursor.fetchall()

conn.close()

df = pd.DataFrame(results, columns=[i[0] for i in cursor.description])
df.head(10)

In [ ]:
df['time_left'].value_counts().plot(kind='bar')

In [ ]:
df = add_features(df)

In [ ]:
print(f'Dataframe shape before duplicates {df.shape}')

df = df.drop_duplicates(subset=['bid_in_gold', 'buyout_in_gold', 'quantity', 'time_left', 'first_appearance_timestamp', 'item_id'])
df = df[(df['time_left'] != 2)]

print(f'Dataframe shape after duplicates {df.shape}')

In [ ]:
X, y = transform_data(df)

df.head(5)

In [ ]:
df.columns

loads a trained model from a file, makes predictions on a data set, calculates the RMSE and displays the result

In [ ]:
pd.options.display.float_format = '{:.0f}'.format

with open('models/forest_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

In [ ]:
predictions = model.predict(X)
df['prediction'] = predictions
df[['item_name', 'item_class', 'unit_price', 'bid_in_gold', 'buyout_in_gold', 'time_left', 'hours_on_sale', 'prediction']].head(10)

In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y, predictions)

print("MAE:", mae)

In [ ]:
df[['item_name', 'item_class', 'unit_price', 'bid_in_gold', 'buyout_in_gold', 'item_id', 'quantity', 'first_appearance_timestamp', 'median_buyout_price',
       'median_bid_price', 'median_unit_price', 'rank_buyout_price',
       'rank_bid_price', 'rank_unit_price', 'avg_competitor_price',
       'std_competitor_price', 'competitor_count', 'lowest_competitor_price',
       'top_competitor_price', 'relative_price_difference',
       'relative_avg_price_difference', 'relative_buyout_difference',
       'relative_bid_difference', 'relative_price_to_lowest_competitor',
       'relative_price_to_top_competitor', 'time_left', 'hours_on_sale', 'prediction']].to_excel('MAE_6_45.xlsx', index=False)

# Error analysis

In [ ]:
df_val = pd.read_excel('MAE_5_54.xlsx')
df_test = pd.read_excel('MAE_6_45.xlsx')

df_val['error'] = np.abs(df_val['hours_on_sale'] - df_val['prediction'])
df_test['error'] = np.abs(df_test['hours_on_sale'] - df_test['prediction'])

df_test.head()

In [ ]:
corr_matrix = df_val[['bid_in_gold', 'buyout_in_gold',
       'unit_price', 'quantity', 'time_left', 'item_id',
       'median_buyout_price', 'median_bid_price', 'median_unit_price',
       'rank_buyout_price', 'rank_bid_price', 'rank_unit_price',
       'avg_competitor_price', 'std_competitor_price', 'competitor_count',
       'lowest_competitor_price', 'top_competitor_price',
       'relative_price_difference', 'relative_avg_price_difference',
       'relative_buyout_difference', 'relative_bid_difference',
       'relative_price_to_lowest_competitor',
       'relative_price_to_top_competitor','hours_on_sale', 'prediction', 'error']].corr()

corr_matrix = corr_matrix[['prediction', 'hours_on_sale', 'error']]

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation with Prediction and Hours on Sale - MAE 5.54')
plt.show()

In [ ]:
corr_matrix = df_test[['bid_in_gold', 'buyout_in_gold',
       'unit_price', 'quantity', 'time_left', 'item_id',
       'median_buyout_price', 'median_bid_price', 'median_unit_price',
       'rank_buyout_price', 'rank_bid_price', 'rank_unit_price',
       'avg_competitor_price', 'std_competitor_price', 'competitor_count',
       'lowest_competitor_price', 'top_competitor_price',
       'relative_price_difference', 'relative_avg_price_difference',
       'relative_buyout_difference', 'relative_bid_difference',
       'relative_price_to_lowest_competitor',
       'relative_price_to_top_competitor','hours_on_sale', 'prediction', 'error']].corr()

corr_matrix = corr_matrix[['prediction', 'hours_on_sale', 'error']]

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation with Prediction and Hours on Sale - MAE 6.45')
plt.show()

In [ ]:
mean_errors_val = df_val.groupby('time_left')['error'].mean().reset_index()
mean_errors_test = df_test.groupby('time_left')['error'].mean().reset_index()

print(mean_errors_val)
print(mean_errors_test)

fig, ax = plt.subplots()

# Create a bar plot with the validation and test bars side by side
bar_width = 0.35
x = np.arange(len(mean_errors_val))
ax.bar(x - bar_width/2, mean_errors_val['error'], bar_width, label='Validation (5.54 MAE)')
ax.bar(x + bar_width/2, mean_errors_test['error'], bar_width, label='Test (6.45 MAE)')

# Set the x-axis tick labels
ax.set_xticks(x)
ax.set_xticklabels(mean_errors_val['time_left'])

plt.xlabel('Time Left')
plt.ylabel('Mean Error')
plt.title('Mean Error by Time Left (Validation vs Test)')
plt.legend()
plt.show()